<a href="https://colab.research.google.com/github/studboii/ML-Colab_Demos/blob/main/Notebooks/MNIST_with_no_ML_and_a_linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!mkdir Data-Sets

In [ ]:
from google.colab import files
uploader = files.upload()

We use 7z to unzip and decompress the archive. -so sends thr output to standard out and the next part untars it and saves it.

In [ ]:
!7z x "/content/MNIST-As-PNG.tar.gz" -so | 7z x -aoa -si -ttar -o"/content/Data-Sets/MNIST-As-PNG"

In [ ]:
path = Path("/content/Data-Sets/MNIST-As-PNG/mnist_png")
Path.BASE_PATH = path

Now we set up a base-line test  by stacking them to see if it would work as i plan on it.

In [ ]:
zeroes = (path/'training'/'0').ls().sorted()
ones = (path/'training'/'1').ls().sorted()
ZeroT = [tensor(Image.open(o)) for o in zeroes]
OneT = [tensor(Image.open(o)) for o in ones]
len(ZeroT),len(OneT)

In [ ]:
show_image(ZeroT[1])

In [ ]:
show_image(OneT[3])

In [ ]:
stack1 = torch.stack(OneT).float()/255
stack0 = torch.stack(ZeroT).float()/255

In [ ]:
stack1.shape,stack0.shape

In [ ]:
mean0 = stack0.mean(0)
mean1 = stack1.mean(0)
show_image(mean0),show_image(mean1)

In [ ]:
Vzeroes = (path/'testing'/'0').ls().sorted()
Vones = (path/'testing'/'1').ls().sorted()
VZeroT = [tensor(Image.open(o)) for o in Vzeroes]
VOneT = [tensor(Image.open(o)) for o in Vones]
len(VZeroT),len(VOneT)

In [ ]:
Vstack1 = torch.stack(VOneT).float()/255
Vstack0 = torch.stack(VZeroT).float()/255

In [ ]:
Vstack1.shape,Vstack0.shape

In [ ]:
def Dist(a,b):
    return (a-b).abs().mean((-1,-2))

In [ ]:
def Is_Zero(x):
    return Dist(x,mean0) < Dist(x,mean1)

In [ ]:
Accu0 = Is_Zero(Vstack0).float().mean()
Accu1 = (1-Is_Zero(Vstack1).float()).mean()
Accu0,Accu1,(Accu0+Accu1)/2

In [ ]:
times = torch.arange(0,30).float()
times

In [ ]:
speed = torch.randn(30)*5 + 0.55*(times-15.5)**2 + 1
plt.scatter(times,speed),

In [ ]:
def f(t, params):
    a,b,c = params
    return a*(t**2) + (b*t) + c

In [ ]:
def mse(preds, targets):
    return ((preds-targets)**2).mean().sqrt()

In [ ]:
params = torch.randn(3).requires_grad_()

In [ ]:
orig_params = params.clone()
orig_params

In [ ]:
preds = f(times, params)
preds.shape, times.shape, speed.shape

In [ ]:
def show_preds(preds,ax=None):
    if ax is None: ax=plt.subplots()[1]
    ax.scatter(times, speed)
    ax.scatter(times, to_np(preds), color='red')
    ax.set_ylim(-300,100)

In [ ]:
show_preds(preds)

In [ ]:
loss = mse(preds, speed)
loss

In [ ]:
loss.backward()
params.grad

Our equivalent of multiplying by a learning rate


In [ ]:
params.grad * 1e-5

In [ ]:
params

In [ ]:
lr = 1e-5
params.data -= lr * params.grad.data
params.grad = None

In [ ]:
preds = f(times,params)

In [ ]:
mse(preds, speed)

In [ ]:
show_preds(preds)

In [ ]:
def apply_step(params, prn=True):
    preds = f(times, params)
    loss = mse(preds, speed)
    loss.backward()
    params.data -= lr * params.grad.data
    params.grad = None
    if prn: print(loss.item())
    return preds

In [ ]:
for i in range(10): apply_step(params)

In [ ]:
params = orig_params.detach().requires_grad_()
params

In [ ]:
def run_epoch(NumOfEpochs):
    for i in range(NumOfEpochs):
        preds = apply_step(params, True)
        show_preds(preds)

In [ ]:
run_epoch(60)

In [ ]:
trainx = torch.cat([stack0,stack1]).view(-1,28*28)
trainy = tensor([1]*len(zeroes) + [0]*len(ones)).unsqueeze(1)
trainx.shape,trainy.shape

In [ ]:
TrainSet = list(zip(trainx,trainy))

In [ ]:
Validx = torch.cat([Vstack0,Vstack1]).view(-1,28*28)
Validy = tensor([1]*len(Vzeroes) + [0]*len(Vones)).unsqueeze(1)
Validx.shape,Validy.shape

In [ ]:
ValidSet = list(zip(Validx,Validy))

In [ ]:
linear_model = nn.Linear(28*28,1)

In [ ]:
w,b = linear_model.parameters()
w.shape,b.shape

In [ ]:
dl = DataLoader(TrainSet, batch_size=256)#x and y combined

In [ ]:
valid_dl = DataLoader(ValidSet, batch_size=256)

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [ ]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD, loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(10, lr=1)

In [ ]:
learn.fit(40,1)

In [ ]:
learn.save("LinearModel")